In [104]:
import pandas as pd
import numpy as np
import numpy_financial as npf

from pyxirr import xirr
from importlib import reload

from str.constantes import com_gst_bco, d_mercado, arancel

import str.cheques as cheques
import str.pagares as pagares
import str.credito as credito
import str.venta   as venta

df, tea, tem = cheques.flujo(0.51, 0.05, 30)
df, tea, tem = pagares.flujo(0.51, 0.05, 30)
df, tea, tem = credito.flujo(tna=1.71, plazo=9, colocacion=0.05, cobranza=0.04, iibb=0.07)
df, tea, tem = venta.flujo(fecha_vta=pd.Period.now('D') + 30, tna_desc=0.6, tna=1.71, plazo=9, colocacion=0.05, cobranza=0.04, iibb=0.07, fondeo=False)
df, tea, tem = venta.flujo(fecha_vta=pd.Period.now('D') + 30, tna_desc=0.6, tna=1.71, plazo=9, colocacion=0.05, cobranza=0.04, iibb=0.07, fondeo=True)

Costo de fondearse con inversores de la BCBB:
    TEM: 5.13%
    TEA: 83.81%
Costo de fondearse con pagares:
    TEM: 6.21%
    TEA: 108.22%
Rendimiento de vender créditos:
    TEM: 6.32%
    TEA: 110.89%
Rendimiento de vender cartera:
    TEM: 15.50%
    TEA: 477.45%
Costo fondearse vendiendo cartera:
    TEM: 6.94%
    TEA: 126.10%


In [103]:
tna = 1.71
plazo = 12

gracia = 2
cap = 10**6
fecha = pd.Period.now('D')

colocacion = 0.05
cobranza = 0.04
iibb = 0.07

fecha_vta = fecha + 30
tna_desc = 0.6

tamar = 0.53
tna_pagare = 0.09
plazo_pagare = 180

df_ctas = credito.nuevo(tna, plazo, gracia, cap, fecha)
df_ctas = df_ctas.reset_index(level='Nro_Cuota')
df_ctas['Cap. + Int.'] = df_ctas[['Capital', 'Interés']].sum(axis=1)
df_ctas['Val. Act.'] = df_ctas.apply(lambda row: row['Cap. + Int.']/(1+tna_desc/365)**(row.name - fecha_vta).n, axis=1) 
val_act = df_ctas['Val. Act.'].sum()

precio = 1 / (1 + (tna/365 * plazo))
der_mer = d_mercado * min(plazo_pagare, 90) / 90 * 1.21
gst_arancel = arancel * min(plazo_pagare, 365) / 365 * 1.21

df_pagare = pd.DataFrame(columns=['Emisión', 'Vto.', 'Capital', 'Descuento', 'Percibido', 'Interés', 'Total'])


def nuevo_pagare(df_pagare: pd.DataFrame, capital: float, emision) -> pd.DataFrame:
    vto_pag = emision + plazo_pagare
    df_pagare.loc[len(df_pagare), :] = pd.Series({
        'Emisión'  : emision,
        'Vto.'     : vto_pag,
        'Capital'  : capital,
        'Descuento': capital * precio,
        'Percibido': capital * precio * (1-der_mer-gst_arancel),
        'Interés'  : capital * precio * tamar/365 * plazo_pagare,
        'Total'    : capital * (1 + precio * tamar/365 * plazo_pagare)})
    
    return df_pagare

df_pagare = nuevo_pagare(df_pagare, val_act, fecha_vta)

df = pd.DataFrame()

df.loc[fecha_vta, 'Flujo'] = df_pagare.at[0, 'Percibido'] * (1-com_gst_bco)  # type: ignore

df_ctas['Bco.']  = df_ctas['Total'] * (1-cobranza) * (1-com_gst_bco)
df_ctas['Bco.'] -= df_ctas['IVA'] * (1+com_gst_bco)
df_ctas['Bco.'] -= df_ctas['Interés'] * iibb * (1+com_gst_bco)

adeudado = df_pagare.at[len(df_pagare)-1, 'Capital']

vto_pag = df_pagare.at[len(df_pagare)-1, 'Vto.']
df.loc[vto_pag, 'Flujo'] = -df_pagare.at[0, 'Total'] * (1+com_gst_bco) + df_ctas.loc[df_ctas.index < vto_pag, 'Bco.'].sum() * (1-com_gst_bco) # type: ignore
df_ctas.loc[df_ctas.index < vto_pag, 'Bco.'] = 0.0

dig = 5
cap_pag = np.ceil(-df.loc[vto_pag, 'Flujo'] / 10**dig) * 10**dig # type: ignore
print(dig, f'$ {cap_pag:,.2f}')
while abs(cap_pag + df.loc[vto_pag, 'Flujo']) > 10**dig: # type: ignore
    dig -= 1
    cap_pag = np.ceil(-df.loc[vto_pag, 'Flujo'] / 10**dig) * 10**dig # type: ignore

df_pagare = nuevo_pagare(df_pagare, cap_pag, vto_pag)
# df.loc[vto_pag, 'Flujo'] += df_pagare.at[len(df_pagare)-1, 'Percibido'] * (1-com_gst_bco) # type: ignore

df_ctas.drop(columns=['Nro_Cuota'], inplace=True)
display(df_ctas.map('$ {:,.2f}'.format))
df_pagare.set_index(['Emisión', 'Vto.'], inplace=True)
display(df_pagare.map('$ {:,.2f}'.format))
print(f'$ {df_ctas['Bco.'].sum():,.2f}')

df.map('$ {:,.2f}'.format)



5 $ 1,100,000.00


,Capital,Interés,IVA,Total,Cap. + Int.,Val. Act.,Bco.
Vto.,,,,,,,
2025-12-28,"$ 36,546.14","$ 116,155.33","$ 24,392.61","$ 177,094.08","$ 152,701.47","$ 143,226.58",$ 0.00
2026-01-28,"$ 41,682.62","$ 111,910.30","$ 23,501.16","$ 177,094.08","$ 153,592.92","$ 136,911.09",$ 0.00
2026-02-28,"$ 47,541.03","$ 107,068.64","$ 22,484.41","$ 177,094.08","$ 154,609.67","$ 130,975.80",$ 0.00
2026-03-28,"$ 54,222.82","$ 101,546.50","$ 21,324.76","$ 177,094.08","$ 155,769.32","$ 126,026.92",$ 0.00
2026-04-28,"$ 61,843.73","$ 95,248.23","$ 20,002.12","$ 177,094.08","$ 157,091.96","$ 120,787.60",$ 0.00
2026-05-28,"$ 70,535.74","$ 88,064.75","$ 18,493.59","$ 177,094.08","$ 158,600.49","$ 116,084.23","$ 144,184.18"
2026-06-28,"$ 80,449.39","$ 79,871.64","$ 16,773.05","$ 177,094.08","$ 160,321.03","$ 111,518.31","$ 146,492.01"
2026-07-28,"$ 91,756.39","$ 70,527.02","$ 14,810.67","$ 177,094.08","$ 162,283.41","$ 107,455.87","$ 149,124.21"
2026-08-28,"$ 104,652.56","$ 59,869.03","$ 12,572.49","$ 177,094.08","$ 164,521.59","$ 103,529.93","$ 152,126.35"


,,Capital,Descuento,Percibido,Interés,Total
Emisión,Vto.,,,,,
2025-11-19,2026-05-18,"$ 1,386,967.19","$ 1,313,143.35","$ 1,295,774.24","$ 343,216.10","$ 1,730,183.29"
2026-05-18,2026-11-14,"$ 1,100,000.00","$ 1,041,450.51","$ 1,027,675.11","$ 272,203.78","$ 1,372,203.78"


$ 1,070,842.98


,Flujo
2025-11-19,"$ 1,287,999.59"
2026-05-18,"$ -1,049,617.55"
